In [ ]:
#import necessary packages
import tensorflow as tf
import datetime, os

from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D

from keras import models

from tensorflow.keras.callbacks import TensorBoard

# starting point 
my_model= models.Sequential()

# Add first convolutional block
my_model.add(Conv2D(16, (3, 3), activation='relu', padding='same', 
                    input_shape=(320,240,3)))
my_model.add(MaxPooling2D((2, 2), padding='same'))

# second block
my_model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), padding='same'))
# third block
my_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), padding='same'))
# fourth block
my_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), padding='same'))

# global average pooling
my_model.add(GlobalAveragePooling2D())
# fully connected layer
my_model.add(Dense(64, activation='relu'))
my_model.add(BatchNormalization())
# make predictions
my_model.add(Dense(2, activation='sigmoid'))


# Show a summary of the model. Check the number of trainable parameters
my_model.summary()

# use early stopping to optimally terminate training through callbacks
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

# save best model automatically
mc= ModelCheckpoint('leafdata/your_model.h5', monitor='val_loss',mode='min', verbose=1, save_best_only=True)
cb_list=[es,mc]


# compile model 
my_model.compile(optimizer='adam', loss='binary_crossentropy', 
                 metrics=['accuracy'])


from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

# set up data generator
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

# get training image batches from the directory
train_generator = data_generator.flow_from_directory(
        'leafdata/Training',
        target_size=(178, 218),
        batch_size=12, shuffle = "True",
        class_mode='categorical')

# get validation image batches from the directory
validation_generator = data_generator.flow_from_directory(
        'leafdata/Validation',
        target_size=(178, 218),
        batch_size=12, shuffle = "True",
        class_mode='categorical')

#tensorboard visualizations
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

#fit model
history = my_model.fit(
        train_generator,
        epochs=30,
        steps_per_epoch=3,
        validation_data=validation_generator,
        validation_steps=3, callbacks=[cb_list, tensorboard_callback])

#evaluate best model
from keras.models import load_model
saved_model = load_model('leafdata/your_model.h5')
test_generator = data_generator.flow_from_directory(
        'leafdata/Testing',
        target_size=(178, 218),
        batch_size=12,
        class_mode='categorical')
saved_model.evaluate(test_generator)